### 影片內容

Tquant介紹

如何用Tquant建立並回測一個策略


Tquant (https://tquant.tejwin.com/)

- Tej維護開發
- 基於zipline的回測引擎
- 避免survivorship bias
- 避免look-ahead bias
- 擬真transaction costs


Tquant安裝 (https://www.youtube.com/watch?v=mmHUBMc3TdM)

Tquant試用 (https://tquant.tejwin.com/shop/)


In [6]:
# Step 1 設定api key
# pip install python-dotenv
import os
import tejapi
from dotenv import load_dotenv

load_dotenv()
tejapi.ApiConfig.api_key = os.environ.get("TEJAPI_KEY")
tejapi.ApiConfig.api_base = os.environ.get("TEJAPI_BASE")

In [21]:
# Goal - 利用EPS分組持有股票並進行回測

In [8]:
# # Step 2 - 設定universe
import pandas as pd
from zipline.sources.TEJ_Api_Data import get_universe

start = '2019-01-01'
end = '2023-04-01'

start_dt = pd.Timestamp(start, tz='utc')
end_dt = pd.Timestamp(end, tz='utc')

tickers = get_universe(start, end, mkt_bd_e=['TSE', 'OTC'], stktp_c=['普通股'])

In [3]:
len(tickers)

1722

In [24]:
# Step 3 - 尋找對應的columns


In [4]:
# Step 4 - 下載外部資料 
import TejToolAPI 
columns = ['eps', 'r104', 'per', 'mktcap']
quality_data = TejToolAPI.get_history_data(ticker=tickers,
                            columns=columns,
                            start=start,
                            end=end)
quality_data['mdate'] = pd.to_datetime(quality_data['mdate'], utc=True)

In [5]:
quality_data.head()

,coid,mdate,Market_Cap_Dollars,PER_TWSE,Return_Rate_on_Equity_A_percent_A,Return_Rate_on_Equity_A_percent_Q,Return_Rate_on_Equity_A_percent_TTM,Basic_Earnings_Per_Share_A,Basic_Earnings_Per_Share_Q,Basic_Earnings_Per_Share_TTM
0,1101,2019-01-02 00:00:00+00:00,1.813361e+11,9.75,9.32,3.37,11.64,3.35,1.25,4.05
1,1101,2019-01-03 00:00:00+00:00,1.795483e+11,9.66,9.32,3.37,11.64,3.35,1.25,4.05
2,1101,2019-01-04 00:00:00+00:00,1.810807e+11,9.74,9.32,3.37,11.64,3.35,1.25,4.05
3,1101,2019-01-07 00:00:00+00:00,1.838902e+11,9.89,9.32,3.37,11.64,3.35,1.25,4.05
4,1101,2019-01-08 00:00:00+00:00,1.815915e+11,9.77,9.32,3.37,11.64,3.35,1.25,4.05


In [6]:
# Step 5 - 儲存外部資料
import sqlite3

conn = sqlite3.connect('factor_database.db')
table_name = 'quality_factors'
quality_data['mdate'] = quality_data['mdate'].astype('datetime64[ns]')
quality_data.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()
# Step 6 - 建立策略
# Step 7 - 下載歷史下載
# Step 8 - 回測


In [5]:
os.environ['mdate'] = start + ' ' + end
os.environ['ticker'] = " ".join(tickers + ['IR0001'])

In [1]:
!zipline ingest -b tquant

In [6]:
# 價量的格式
from zipline.data import bundles
from zipline.data.data_portal import get_bundle

bundle_name = 'tquant'
bundle = bundles.load(bundle_name)

df_bundle = get_bundle(bundle_name=bundle_name,
                       calendar_name='TEJ',
                       start_dt=start_dt,
                       end_dt=end_dt)
df_bundle

,date,sid,symbol,asset,open,high,low,close,volume,open_adj,...,close_adj,volume_adj,dividend_payouts.amount,dividend_payouts.declared_date,dividend_payouts.div_percent,dividend_payouts.pay_date,dividend_payouts.record_date,dividends.ratio,splits.ratio,mergers.ratio
0,2019-01-03 00:00:00+00:00,0,1101,Equity(0 [1101]),35.50,35.60,35.10,35.15,9160000.0,22.918,...,22.692,1.131921e+07,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN
1,2019-01-03 00:00:00+00:00,1,1102,Equity(1 [1102]),33.95,34.15,33.75,33.90,3918000.0,25.728,...,25.690,3.918000e+06,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN
2,2019-01-03 00:00:00+00:00,2,1103,Equity(2 [1103]),13.60,13.65,13.55,13.60,114000.0,10.888,...,10.888,1.140000e+05,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN
3,2019-01-03 00:00:00+00:00,3,1104,Equity(3 [1104]),18.95,18.95,18.70,18.90,72000.0,15.473,...,15.432,7.200000e+04,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN
4,2019-01-03 00:00:00+00:00,4,1108,Equity(4 [1108]),7.34,7.44,7.29,7.40,132000.0,6.490,...,6.543,1.320000e+05,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776408,2023-04-06 00:00:00+00:00,1718,9955,Equity(1718 [9955]),NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,0.000000e+00,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN
1776409,2023-04-06 00:00:00+00:00,1719,9958,Equity(1719 [9958]),NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,0.000000e+00,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN
1776410,2023-04-06 00:00:00+00:00,1720,9960,Equity(1720 [9960]),NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,0.000000e+00,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN
1776411,2023-04-06 00:00:00+00:00,1721,9962,Equity(1721 [9962]),NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,0.000000e+00,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN


In [7]:
from zipline.pipeline.data.dataset import Column, DataSet
from zipline.pipeline.domain import TW_EQUITIES

class CustomDataset(DataSet):
    Basic_Earnings_Per_Share_TTM = Column(dtype=float)
    PER_TWSE = Column(dtype=float)
    Market_Cap_Dollars = Column(dtype=float)
    domain = TW_EQUITIES

In [8]:
sids = bundle.asset_finder.equities_sids
assets = bundle.asset_finder.retrieve_all(sids)
symbol_mapping_sid = {i.symbol: i.sid for i in assets}
transform_data = quality_data.set_index(['coid', 'mdate']).unstack('coid')
transform_data = transform_data.rename(columns=symbol_mapping_sid)
transform_data

Market_Cap_Dollars                              \
coid                                    0             1             2      
mdate                                                                      
2019-01-02 00:00:00+00:00       1.813361e+11  1.136169e+11  1.053702e+10   
2019-01-03 00:00:00+00:00       1.795483e+11  1.139531e+11  1.053702e+10   
2019-01-04 00:00:00+00:00       1.810807e+11  1.137850e+11  1.053702e+10   
2019-01-07 00:00:00+00:00       1.838902e+11  1.171464e+11  1.073071e+10   
2019-01-08 00:00:00+00:00       1.815915e+11  1.164741e+11  1.073071e+10   
...                                      ...           ...           ...   
2023-03-27 00:00:00+00:00       2.622741e+11  1.521203e+11  1.475957e+10   
2023-03-28 00:00:00+00:00       2.590538e+11  1.528295e+11  1.460461e+10   
2023-03-29 00:00:00+00:00       2.601272e+11  1.535387e+11  1.464335e+10   
2023-03-30 00:00:00+00:00       2.590538e+11  1.531841e+11  1.468209e+10   
2023-03-31 00:00:00+00:00       2.604850e+11  1.535387e+11  1.468209e+10   

                                                                     \
coid                               3             4             5      
mdate                                                                 
2019-01-02 00:00:00+00:00  1.235321e+10  2.999109e+09  4.616431e+09   
2019-01-03 00:00:00+00:00  1.235321e+10  2.995061e+09  4.599333e+09   
2019-01-04 00:00:00+00:00  1.225517e+10  2.999109e+09  4.633529e+09   
2019-01-07 00:00:00+00:00  1.232053e+10  2.999109e+09  4.616431e+09   
2019-01-08 00:00:00+00:00  1.225517e+10  2.999109e+09  4.616431e+09   
...                                 ...           ...           ...   
2023-03-27 00:00:00+00:00  1.715724e+10  5.706806e+09  5.799701e+09   
2023-03-28 00:00:00+00:00  1.699384e+10  5.666332e+09  5.816759e+09   
2023-03-29 00:00:00+00:00  1.709188e+10  5.625858e+09  5.816759e+09   
2023-03-30 00:00:00+00:00  1.705920e+10  5.747280e+09  5.816759e+09   
2023-03-31 00:00:00+00:00  1.732064e+10  5.727043e+09  5.816759e+09   

                                                                     \
coid                               6             7             8      
mdate                                                                 
2019-01-02 00:00:00+00:00  9.609613e+09  1.113338e+10  6.240000e+09   
2019-01-03 00:00:00+00:00  9.380813e+09  1.115869e+10  6.288000e+09   
2019-01-04 00:00:00+00:00  9.438013e+09  1.110808e+10  6.288000e+09   
2019-01-07 00:00:00+00:00  9.609613e+09  1.146232e+10  6.240000e+09   
2019-01-08 00:00:00+00:00  9.409413e+09  1.125990e+10  6.276000e+09   
...                                 ...           ...           ...   
2023-03-27 00:00:00+00:00  1.012441e+10  1.012126e+10  9.216000e+09   
2023-03-28 00:00:00+00:00  1.009581e+10  1.009596e+10  9.252000e+09   
2023-03-29 00:00:00+00:00  1.006721e+10  1.007065e+10  9.324000e+09   
2023-03-30 00:00:00+00:00  1.006721e+10  1.009596e+10  9.264000e+09   
2023-03-31 00:00:00+00:00  1.003861e+10  1.012126e+10  9.300000e+09   

                                         ...  \
coid                               9     ...   
mdate                                    ...   
2019-01-02 00:00:00+00:00  2.663244e+10  ...   
2019-01-03 00:00:00+00:00  2.686882e+10  ...   
2019-01-04 00:00:00+00:00  2.742038e+10  ...   
2019-01-07 00:00:00+00:00  2.773556e+10  ...   
2019-01-08 00:00:00+00:00  2.738098e+10  ...   
...                                 ...  ...   
2023-03-27 00:00:00+00:00  4.491732e+10  ...   
2023-03-28 00:00:00+00:00  4.500679e+10  ...   
2023-03-29 00:00:00+00:00  4.491732e+10  ...   
2023-03-30 00:00:00+00:00  4.500679e+10  ...   
2023-03-31 00:00:00+00:00  4.572260e+10  ...   

                          Return_Rate_on_Equity_A_percent_TTM               \
coid                                                     1712   1713  1714   
mdate                                                                        
2019-01-02 00:00:00+00:00                                0.80  3

In [9]:
from zipline.pipeline.loaders.frame import DataFrameLoader

inputs = [CustomDataset.Basic_Earnings_Per_Share_TTM,
          CustomDataset.PER_TWSE,
          CustomDataset.Market_Cap_Dollars]
custom_loader = {
    i:
    DataFrameLoader(column=i,
                    baseline=transform_data.xs(i.name, level=0, axis=1))
    for i in inputs
}

In [10]:
from strategy import PercentageIndicatorStrategy
from zipline.pipeline import Pipeline


class EpsPercentageIndicatorStrategy(PercentageIndicatorStrategy):

    def compute_signals(self):
        BE = CustomDataset.Basic_Earnings_Per_Share_TTM.latest
        print(f"Basic Earnings Per Share: {BE}")
        top_percentile = CustomDataset.Basic_Earnings_Per_Share_TTM.latest.percentile_between(
            80, 100)
        # create pipeline
        pipe = Pipeline(columns={
            'Basic_Earnings_Per_Share_TTM': BE,
            'longs': top_percentile
        })
        print(f"Pipeline columns: {pipe.columns}")
        return pipe
    

In [3]:
import matplotlib.pyplot as plt
from zipline import run_algorithm
from zipline.utils.calendar_utils import get_calendar


def initialize(context):
    context.strategy = EpsPercentageIndicatorStrategy(assets, start_dt, end_dt)
    context.strategy.initialize(context)


def before_trading_start(context, data):
    context.strategy.before_trading_start(context, data)


def analyze(context, perf):

    fig = plt.figure(figsize=(16, 12))

    # First chart(累計報酬)
    ax = fig.add_subplot(311)
    ax.set_title('Strategy Results')
#     ax.plot(perf['algorithm_period_return'],
            linestyle='-',
            label='algorithm period return',
            linewidth=3.0)
    ax.plot(perf['benchmark_period_return'],
            linestyle='-',
            label='benchmark period return',
            linewidth=3.0)
    ax.legend()
    ax.grid(False)
    # Second chart(ending_cash)->觀察是否超買
    ax = fig.add_subplot(312)
    ax.plot(perf['ending_cash'],
            label='ending_cash',
            linestyle='-',
            linewidth=1.0)
    ax.axhline(y=1, c='r', linewidth=0.3)
    ax.legend()
    ax.grid(True)


result = run_algorithm(start=start_dt,
                       end=end_dt,
                       initialize=initialize,
                       before_trading_start=before_trading_start,
                       capital_base=1e6,
                       data_frequency='daily',
                       bundle='tquant',
                       analyze=analyze,
                       trading_calendar=get_calendar("TEJ"),
                       custom_loader=custom_loader)


In [4]:
report_table = result.T.copy()
report_table.loc[:, (report_table.columns >= '2019-01-15')]

In [5]:
import pyfolio
from pyfolio.utils import extract_rets_pos_txn_from_zipline

returns, positions, transactions = extract_rets_pos_txn_from_zipline(result)
benchmark_rets = result.benchmark_return
pyfolio.tears.create_full_tear_sheet(returns=returns,
                                     positions=positions,
                                     transactions=transactions,
                                     benchmark_rets=benchmark_rets
                                    )
